# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [22]:
# Write your code below.

import os
PRICE_DATA = os.getenv("PRICE_DATA")

In [23]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [25]:
import os
from glob import glob

# Write your code below.
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=64,,,,,,,,
AAPL,datetime64[ns],float64,float64,float64,float64,float64,float64,int32
ACN,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [51]:
# Write your code below.
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Close'].shift(1))
)
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag_1 = x['Adj Close'].shift(1))
)

# Return based on Close.
dd_rets = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1
)

C:\Users\dimoz\AppData\Local\Temp\ipykernel_23680\1452606604.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
C:\Users\dimoz\AppData\Local\Temp\ipykernel_23680\1452606604.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [52]:
dd_rets

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Returns
npartitions=63,,,,,,,,,,
AAPL,datetime64[ns],float64,float64,float64,float64,float64,float64,int32,float64,float64
ACN,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
WDC,...,...,...,...,...,...,...,...,...,...
ZBRA,...,...,...,...,...,...,...,...,...,...


In [53]:
# Write your code below.
dd_rets = dd_rets.compute()

In [54]:
print(dd_rets)

             Date   Adj Close       Close        High         Low        Open  \
Ticker                                                                          
AAPL   2000-01-03    0.843076    0.999442    1.004464    0.907924    0.936384   
AAPL   2000-01-04    0.771997    0.915179    0.987723    0.903460    0.966518   
AAPL   2000-01-05    0.783293    0.928571    0.987165    0.919643    0.926339   
AAPL   2000-01-06    0.715509    0.848214    0.955357    0.848214    0.947545   
AAPL   2000-01-07    0.749401    0.888393    0.901786    0.852679    0.861607   
...           ...         ...         ...         ...         ...         ...   
ZBRA   2025-01-17  405.709991  405.709991  407.290009  402.290009  406.040009   
ZBRA   2025-01-21  418.070007  418.070007  419.850006  407.619995  407.619995   
ZBRA   2025-01-22  420.570007  420.570007  427.760010  419.589996  425.239990   
ZBRA   2025-01-23  421.109985  421.109985  422.290009  414.450012  417.619995   
ZBRA   2025-01-24  414.60998

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
```
No, it was not necessary
```
+ Would it have been better to do it in Dask? Why?
```
Yes, it would have been better to calculate the moving average in Dask rather than converting to Pandas, especially if the dataset is large. Here is why:
 a) Dask is designed to handle large datasets that don't fit into memory
 b) Dask allows parallel computation across multiple cores or even distributed computing clusters
 c) Dask operates lazily, meaning it doesn't load the entire dataset into memory at once
 d) Converting from Dask to Pandas can introduce unnecessary overhead, especially with large datasets
```
(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.